In [1]:
import requests
import pandas as pd
import time

# 네이버 API 인증 정보 
client_id = " "
client_secret = " "

# 네이버 블로그 검색 함수
def naver_blog_search(query, display=100, start=1):
    """
    네이버 블로그 검색 API를 호출하여 데이터를 가져옵니다.
    Parameters:
        query (str): 검색 키워드
        display (int): 한 번에 가져올 결과 개수 (1~100)
        start (int): 검색 시작 위치 (1~1000)
    Returns:
        list: 검색 결과를 포함하는 딕셔너리 리스트
    """
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": query,
        "display": display,
        "start": start
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json().get("items", [])
    else:
        print(f"Error: {response.status_code}")
        return []

# 특정 키워드를 조합하여 블로그 데이터를 가져오는 함수
def collect_combined_blog_data(restaurant, program):
    """
    식당 이름과 출연 프로그램 키워드를 조합하여 네이버 블로그 데이터를 수집합니다.
    Parameters:
        restaurant (str): 식당 이름
        program (str): 출연 프로그램 키워드
    Returns:
        DataFrame: 조합된 키워드로 검색된 블로그 데이터를 포함하는 데이터프레임
    """
    query = f"{restaurant} {program}"  # 두 키워드를 조합하여 검색
    all_data = []
    start = 1
    while True:
        items = naver_blog_search(query, display=100, start=start)
        if not items:  # 더 이상 데이터가 없으면 종료
            break
        
        # 데이터 수집
        for item in items:
            all_data.append({
                "title": item["title"],  # 블로그 제목
                "link": item["link"],    # 블로그 URL
                "description": item["description"],  # 설명
                "blogger_name": item["bloggername"], # 블로거 이름
                "post_date": item["postdate"]        # 게시 날짜
            })
        
        start += 100  # 다음 100개로 이동
        time.sleep(1)  # 요청 간 대기 (너무 빠른 요청 방지)
    
    return pd.DataFrame(all_data)

# 식당과 프로그램 정보를 기반으로 리뷰 데이터 수집
def collect_all_filtered_data(restaurant_program_dict):
    """
    식당 이름과 출연 프로그램 키워드로 모든 리뷰 데이터를 수집합니다.
    Parameters:
        restaurant_program_dict (dict): 식당 이름과 프로그램 키워드 딕셔너리
    Returns:
        DataFrame: 모든 조합된 키워드의 리뷰 데이터를 포함한 데이터프레임
    """
    all_filtered_data = []
    
    for restaurant, program in restaurant_program_dict.items():
        print(f"Collecting reviews for: {restaurant} ({program})")
        filtered_data = collect_combined_blog_data(restaurant, program)
        filtered_data["restaurant_name"] = restaurant  # 식당 이름 추가
        filtered_data["program_name"] = program        # 출연 프로그램 추가
        all_filtered_data.append(filtered_data)
    
    # 모든 데이터를 하나의 DataFrame으로 합치기
    combined_data = pd.concat(all_filtered_data, ignore_index=True)
    return combined_data

'''
# 식당과 출연 프로그램 정보를 딕셔너리로 정리
restaurant_program_dict = {
    "시장횟집": "골목식당",
    "준까스": "골목식당",
    "온전히": "골목식당",
    "필동멸치국수": "골목식당",
    "소년아저씨": "골목식당",
    "맹그로브": "골목식당",
    "단초식당 선릉점": "골목식당",
    "더리버 벨라쿠치나": "맛있는 녀석들",
    "오후청": "맛있는 녀석들",
    "초량돼지": "맛있는 녀석들",
    "청담 테라": "수요미식회",
    "코퍼룸": "수요미식회",
    "순수식판": "또간집",
    "더테이블키친": "또간집",
    "이유식당": "또간집",
    "민어집 병우네": "수요미식회",
    "우일집": "6시내고향",
    "금복족발": "생활의달인",
    "낀 알로이 아로이": "생활의달인",
    "소바야린": "골목식당",
    "봄플레이트": "골목식당",
    "경복식당": "골목식당",
    "어머니와아들": "골목식당",
    "솜이네떡볶이": "골목식당",
    "필동함박": "골목식당",
    "은희네온집닭떡볶이": "골목식당",
    "김여사네국수": "골목식당",
    "맛있는식사": "골목식당",
    "송죽장": "맛있는 녀석들",
    "메이드인시카고피자": "맛있는 녀석들",
    "포썸": "맛있는 녀석들",
    "툇마루밥상": "수요미식회",
    "피자힐": "수요미식회",
    "유야케도쿄": "또간집",
    "오복순대국": "또간집",
    "산청엔흑돼지": "또간집",
    "대관원": "수요미식회",
    "계동피자": "6시내고향",
    "신촌즉석생우동": "생활의달인",
    "피자리움": "생활의달인",
    "오복함흥냉면": "골목식당"
}
'''


'\n# 식당과 출연 프로그램 정보를 딕셔너리로 정리\nrestaurant_program_dict = {\n    "시장횟집": "골목식당",\n    "준까스": "골목식당",\n    "온전히": "골목식당",\n    "필동멸치국수": "골목식당",\n    "소년아저씨": "골목식당",\n    "맹그로브": "골목식당",\n    "단초식당 선릉점": "골목식당",\n    "더리버 벨라쿠치나": "맛있는 녀석들",\n    "오후청": "맛있는 녀석들",\n    "초량돼지": "맛있는 녀석들",\n    "청담 테라": "수요미식회",\n    "코퍼룸": "수요미식회",\n    "순수식판": "또간집",\n    "더테이블키친": "또간집",\n    "이유식당": "또간집",\n    "민어집 병우네": "수요미식회",\n    "우일집": "6시내고향",\n    "금복족발": "생활의달인",\n    "낀 알로이 아로이": "생활의달인",\n    "소바야린": "골목식당",\n    "봄플레이트": "골목식당",\n    "경복식당": "골목식당",\n    "어머니와아들": "골목식당",\n    "솜이네떡볶이": "골목식당",\n    "필동함박": "골목식당",\n    "은희네온집닭떡볶이": "골목식당",\n    "김여사네국수": "골목식당",\n    "맛있는식사": "골목식당",\n    "송죽장": "맛있는 녀석들",\n    "메이드인시카고피자": "맛있는 녀석들",\n    "포썸": "맛있는 녀석들",\n    "툇마루밥상": "수요미식회",\n    "피자힐": "수요미식회",\n    "유야케도쿄": "또간집",\n    "오복순대국": "또간집",\n    "산청엔흑돼지": "또간집",\n    "대관원": "수요미식회",\n    "계동피자": "6시내고향",\n    "신촌즉석생우동": "생활의달인",\n    "피자리움": "생활의달인",\n    "오복함흥냉면": "골목식당"\n}\n'

In [10]:
review_list = pd.read_excel("C:/Users/rkdgu/DScover/메인프로젝트/추가 리스트2.xlsx",header=None)

final = review_list.iloc[339:]

In [11]:
# 두 번째 열과 세 번째 열을 딕셔너리로 변환
column_dict = dict(zip(final.iloc[:, 2], final.iloc[:, 1]))

# 결과 출력
column_dict

{'청학동부침개 마포본점': '6시내고향',
 '마포소문난원조족발': '6시내고향',
 '장군굴보쌈': '6시내고향',
 '쭈먹': '6시내고향',
 '이포어묵': '6시내고향',
 '라까예': '6시내고향',
 '남매집': '6시내고향',
 '수제비먹는닭갈비': '6시내고향',
 '충무로쭈꾸미불고기 충무로본점': '6시내고향',
 '중림장설렁탕': '6시내고향',
 '이화감자국': '6시내고향',
 '할범탕수육': '6시내고향',
 '안동집 손칼국시': '6시내고향',
 '최루탄해장라면 훼드라': '6시내고향',
 '창신육회 본점': '6시내고향',
 '라화쿵부': '6시내고향',
 '육덕등심 본점': '6시내고향',
 '반피차이': '6시내고향',
 '삼거리먼지막순대국': '6시내고향',
 '바삭마차': '6시내고향',
 '부산갈매기': '6시내고향',
 '원조마포할머니빈대떡': '6시내고향',
 '성북동돼지갈비': '6시내고향',
 '여기가전집': '6시내고향',
 '본점최대포': '6시내고향',
 '능라밥상': '6시내고향',
 '도이칠란드 박': '6시내고향'}

In [12]:
# 모든 조합된 키워드로 리뷰 데이터 수집
filtered_reviews_data = collect_all_filtered_data(column_dict)
filtered_reviews_data


Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400


,title,link,description,blogger_name,post_date,restaurant_name,program_name
0,<b>6시 내고향</b> 맛나는 거리 서울... <b>청학동부침개</b> 족발 맛집...,https://blog.naver.com/pirin888/223410365465,<b>6시 내고향</b> ▽ [<b>6시 내고향</b>]맛나는 거리 서울 <b>마포...,rin,20240409,청학동부침개 마포본점,6시내고향
1,<b>6시내고향</b> 서울 <b>마포</b> 방송정보... 가게 &lt;<b>청학...,https://blog.naver.com/khtd1831/223410506085,"<b>6시 내고향</b> 연출 미등록 출연 윤인구, 가애란, 김영옥, 김진희, 이금...",시네마천국,20240409,청학동부침개 마포본점,6시내고향
2,<b>마포</b> 공덕 맛집 공덕시장 전집 &lt;<b>청학동부침개 마포본점</b>...,https://blog.naver.com/seulog__/223411442587,안녕하세요 슬콩입니다 친구들과 비오는 날 방문했던 #<b>청학동부침개마포본점</b>...,Seulog,20240410,청학동부침개 마포본점,6시내고향
3,비 오는 날 생각나는 공덕 전골목 맛집 <b>청학동부침개 마포본점</b>,https://blog.naver.com/dollvely_/223387899909,맛집 <b>청학동부침개 마포본점</b>에 다녀왔다! 주말 저녁 이른시간임에도 불구하...,돌블리 SHARE-log,20240319,청학동부침개 마포본점,6시내고향
4,[공덕역/<b>마포</b>] 막걸리 천국! '<b>청학동부침개 마포본점</b>',https://blog.naver.com/blooming_time/223325723858,<b>청학동부침개 마포본점</b> 서울시 <b>마포</b>구 만리재로 23 공덕시장...,소소(笑笑)한 기록,20240117,청학동부침개 마포본점,6시내고향
...,...,...,...,...,...,...,...
14005,KBS<b>6시내고향</b> 한국SNS사진명소맛집핫플여행지...,https://blog.naver.com/hsc660312/223342055522,"SBS생방송투데이,KBS <b>6시내고향</b> TV방송에 나온 맛집,핫플여행지,캠...",방송리뷰 드라마 촬영지 정보,20240202,도이칠란드 박,6시내고향
14006,"<b>6시내고향</b> 서울 성북정릉시장 감자탕 떡갈비, 도라지정과, 수제...",https://blog.naver.com/brjang71/223342044431,(*배송비 별도) *시청자 상담실(02-781-1000)을 통해 개별 문의처로 안내...,하루하루를 소중하게,20240202,도이칠란드 박,6시내고향
14007,<b>6시내고향</b> 서울 정릉시장 감자탕 떡갈비 도라지정과...,https://blog.naver.com/shoot8282/223342131353,<b>6시내고향</b> 정릉시장 감자탕 떡갈비 도라지정과 수제 소시지 순대 택배 주...,슈퍼맨의 흔적 남기기,20240202,도이칠란드 박,6시내고향
14008,[정릉] 수제소세지 n번째 방문한 숨은 찐맛집 + 한옥 테라스까지,https://blog.naver.com/merongsnail/223367786771,<b>도이칠란드박</b> 안녕하세요. 달팽이입니다 제가 음 .. 저의 소중한 찐 n...,메롱달팽이 여행일지,20240228,도이칠란드 박,6시내고향


In [13]:
# 데이터 저장 (Excel 파일로 저장)
filtered_reviews_data.to_excel("12_24_blog_reviews추가.xlsx", index=False)
print("Filtered data collection complete!")


Filtered data collection complete!


In [7]:
filtered_reviews_data

,title,link,description,blogger_name,post_date,restaurant_name,program_name
0,이태원 해방촌 신흥<b>시장</b> <b>골목식당</b> <b>시장횟집</b> 알탕,https://blog.naver.com/kidariblue/223041845129,그냥 나왔다 #이태원 #해방촌 #해방촌신흥시장 #해방촌골목식당 #신흥시장골목식당 #...,키다리블루의 블로그,20230312,시장횟집,골목식당
1,백종원의 <b>골목식당</b> 해방촌 <b>시장횟집</b> 2호점 묵동 맛집 동태탕 배달,https://blog.naver.com/ssjrjy/223122818320,마지막 한방울도 남기지 않고 클리어 한 백종원의 <b>골목식당</b> 해방촌<b>시...,S의 먹고놀기,20230607,시장횟집,골목식당
2,백종원의 <b>골목식당</b>에 나온 해방촌 <b>시장횟집</b>에서 시원한 알탕...,https://blog.naver.com/kjd927/223036484482,ㅎㅎ♀️♀️ #해방촌 #<b>시장횟집</b> #이태원 #백종원<b>골목식당</b> ...,행복한 덤선생 일상:),20230306,시장횟집,골목식당
3,해방촌 맛집인 백종원의 <b>골목식당</b> <b>시장횟집</b>을 방문했다.,https://blog.naver.com/prhknw4794/223405859357,<b>시장횟집</b>은 해방촌의 작은 <b>골목</b>에 위치해 있어서 찾기 어려울...,prhknw4794님의 블로그,20240404,시장횟집,골목식당
4,알탕 국물 참 맛있쥬? 백종원 강추 해방촌 <b>골목식당</b> <b>시장횟집</b>,https://blog.naver.com/xhdtls119/223035195795,"백종원 <b>골목식당</b>에 나온 맛집, 해방촌 신흥<b>시장</b> 알탕 &am...",yummy story,20230305,시장횟집,골목식당
...,...,...,...,...,...,...,...
29647,"[용인/수지] <b>냉면</b>도 갈비찜도 맛있는, ‘<b>함흥냉면</b>’",https://blog.naver.com/kmj8884/223455706048,<b>함흥냉면</b> 위치 : 경기 용인시 수지구 풍덕천로 115-22(수지구청역 ...,가족의 일상과 세금 이야기,20240523,오복함흥냉면,골목식당
29648,청파동 <b>냉면</b>집 <b>오복함흥냉면</b> 두번째 먹음. 주차는 이곳에,https://blog.naver.com/5sa11/221460238899,<b>골목식당</b> 청파동 <b>냉면</b>집으로 나왔을 때 방송되자마자 12월 ...,상암여자의 내맘대로블로그,20190207,오복함흥냉면,골목식당
29649,용산 숙대입구역 맛집 <b>골목식당</b> <b>오복함흥냉면</b>집 방문기,https://blog.naver.com/tomboi76/221552781213,ㅎㅎ 백종원의 <b>골목식당</b>에 나온 <b>함흥냉면</b>집이라네요~ wow ...,열정노마드의 드림라이프 Follow your dream!,20190602,오복함흥냉면,골목식당
29650,"청년다방 오징어떡볶이, 용산 <b>오복함흥냉면</b>:)",https://blog.naver.com/e__1234/222718550644,더워지면 생각나는 <b>냉면</b> <b>냉면</b>도 맛있지만 달콤한 비빔<b>냉...,망고코코,20220502,오복함흥냉면,골목식당
